In [15]:
import pandas as pd
import numpy as np
from neo4j.v1 import GraphDatabase

In [ ]:
driver = GraphDatabase.driver("bolt://52.23.222.222:33173/db/data/", 
                              auth=("neo4j", "afternoon-buffer-crusts"))

In [14]:
def add_customer(tx, customer_id, first_name, last_name):
    tx.run("CREATE (a:Customer {id: $customer_id, first_name: $first_name, last_name: $last_name})",
           customer_id=customer_id, first_name=first_name, last_name=last_name)

In [21]:
def add_item(tx, item_id, title, price):
    tx.run("CREATE (a:Item {id: $item_id, title: $title, price: $price})",
           item_id=item_id, title=title, price=price)

In [42]:
def add_order(tx, order_id, customer_id):
    tx.run("CREATE (order:Order {id: $order_id})",
           order_id=order_id)
    tx.run("MATCH (buyer:Customer),(order:Order)"
           "WHERE buyer.id = $customer_id AND order.id = $order_id "
           "CREATE (buyer)-[:MADE]->(order)",
           order_id=order_id, customer_id=customer_id)

In [50]:
def add_order_item(tx, order_id, item_id):
        tx.run("MATCH (item:Item),(order:Order)"
           "WHERE item.id = $item_id AND order.id = $order_id "
           "CREATE (order)-[:CONTAINS]->(item)",
           order_id=order_id, item_id=item_id)

In [55]:
def add_view_item(tx, customer_id, item_id, times_count):
        tx.run("MATCH (i:Item), (c:Customer)"
           "WHERE i.id = $item_id AND c.id = $customer_id "
           "CREATE (c)-[:VIEWED{times:$times_count}]->(i)",
           customer_id=customer_id, item_id=item_id, times_count=times_count)

In [23]:
df_customers = pd.read_csv('customers.csv')
df_items = pd.read_csv('items.csv')
df_orders = pd.read_csv('orders.csv')
df_orders_detail = pd.read_csv('order_details.csv')
df_items_viewed = pd.read_csv('items_viewed.csv')

In [48]:
for index_item, row_item in df_items.iterrows():
    item_id = row_item["id"]
    title = row_item["title"]
    price = row_item["price"]
    with driver.session() as session:
        session.write_transaction(add_item, item_id, title, price)

In [34]:
for index_customer, row_customer in df_customers.iterrows():
    customer_id = row_customer["id"]
    first_name = row_customer["first_name"]
    last_name = row_customer["last_name"]
    with driver.session() as session:
        session.write_transaction(add_customer, customer_id, first_name, last_name)

In [41]:
for index_order, row_order in df_orders.iterrows():
    order_id = row_order["id"]
    customer_id = row_order["customer_id"]
    with driver.session() as session:
        session.write_transaction(add_order, order_id, customer_id)

In [56]:
for index_item_view, row_item_view in df_items_viewed.iterrows():
    customer_id = int(row_item_view["customer_id"])
    item_id = int(row_item_view["item_id"])
    times_count = int(row_item_view["times_count"])
    with driver.session() as session:
        session.write_transaction(add_view_item, customer_id, item_id, times_count)

In [ ]:
for index_order_item, row_order_item in df_orders_detail.iterrows():
    order_id = int(row_order_item["order_id"])
    item_id = int(row_order_item["item_id"])
    with driver.session() as session:
        session.write_transaction(add_order_item, order_id, item_id)

In [ ]:
def add_friends(tx, name, friend_name):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})",
           name=name, friend_name=friend_name)

In [ ]:
def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                         "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])

In [ ]:
def clear_db(tx):
    tx.run("MATCH (n) "
           "DETACH DELETE n")

In [ ]:
with driver.session() as session:
    session.write_transaction(add_friends, "Arthur", "Guinevere")
    session.write_transaction(add_friends, "Arthur", "Lancelot")
    session.write_transaction(add_friends, "Arthur", "Merlin")
    session.read_transaction(print_friends, "Arthur")

In [33]:
with driver.session() as session:
    session.write_transaction(clear_db)